### Exemplo de Prompt para Inferência

Determine cinco tópicos que estão sendo discutidos no texto a seguir, que é delimitado por ####.

Faça cada item ter uma ou duas palavras.

Formate sua resposta como uma lista de itens separados por vírgulas.

####

Em uma pesquisa recente conduzida pelo governo, os funcionários do setor público foram solicitados a avaliar seu nível de satisfação com o departamento em que trabalham.

Os resultados revelaram que a NASA foi o departamento mais popular, com uma taxa de satisfação de 95%.

Um funcionário da NASA, John Smith, comentou sobre os resultados, afirmando:  
"Não estou surpreso que a NASA tenha se destacado.  
É um ótimo lugar para trabalhar, com pessoas incríveis e oportunidades incríveis.  
Tenho orgulho de fazer parte de uma organização tão inovadora."

Os resultados também foram bem recebidos pela equipe de gerenciamento da NASA, com o diretor Tom Johnson afirmando:  
"Estamos emocionados ao saber que nossos funcionários estão satisfeitos com seu trabalho na NASA.  
Temos uma equipe talentosa e dedicada que trabalha incansavelmente para alcançar nossos objetivos,  
e é fantástico ver que seu trabalho árduo está dando frutos."

A pesquisa também revelou que a Administração da Previdência Social teve a menor pontuação de satisfação,  
com apenas 45% dos funcionários indicando que estavam satisfeitos com seus empregos.

O governo se comprometeu a abordar as preocupações levantadas pelos funcionários na pesquisa  
e trabalhar para melhorar a satisfação no trabalho em todos os departamentos.

####


### Exemplo de Prompt para Treinar LangChain com Schema

Extraia do texto a seguir as seguintes informações no formato JSON:

- **nome_evento**: Nome do evento.
- **data**: Data do evento.
- **local**: Cidade e país onde ocorrerá.
- **organizadores**: Lista de organizadores.
- **temas**: Lista de temas abordados.
- **valor_ingresso**: Preço do ingresso em moeda local.

Texto:


O *Tech Future Conference 2025* acontecerá nos dias **14 e 15 de setembro de 2025**, em **Lisboa, Portugal**.  
O evento será organizado pela **Future Minds** em parceria com a **InovaTech** e reunirá especialistas de várias áreas para discutir temas como **Inteligência Artificial**, **Blockchain**, **Computação Quântica** e **Tecnologias Sustentáveis**.  
Os ingressos já estão à venda por **€249** e podem ser adquiridos no site oficial do evento.  
A conferência contará com palestrantes internacionais, workshops práticos e sessões de networking voltadas para startups e investidores.


In [1]:
# import libs 
from langchain_google_genai import ChatGoogleGenerativeAI #lib para gemini
from langchain.prompts import ChatPromptTemplate  # lib para prompt
from dotenv import load_dotenv # lib para acessar .env e chave API da Gemini
from langchain.output_parsers import ResponseSchema, StructuredOutputParser # libs para OutPut Parses

load_dotenv() #carrega arquivo .env e pega a chave API


c:\Users\user\Documents\Python\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
# Texto para extrair

Texto_evento = ' O Tech Future Conference 2025 acontecerá nos dias 14 e 15 de setembro de 2025, em Lisboa, Portugal. ' \
'O evento será organizado pela Future Minds em parceria com a InovaTech e reunirá especialistas de várias áreas para ' \
'discutir temas como Inteligência Artificial, Blockchain, Computação Quântica e Tecnologias Sustentáveis. ' \
'Os ingressos já estão à venda por €249 e podem ser adquiridos no site oficial do evento.A conferência contará com ' \
'palestrantes internacionais, workshops práticos e sessões de networking voltadas para startups e investidores.' 

In [3]:
# criando Schema

schema_evento = ResponseSchema(
    name='nome_evento',
    type='str',
    description='Nome do evento em caso de mais de uma, separar por vírgula, caso não tenha nenhuma colocar como unterfuned'
)

schema_data = ResponseSchema(
    name='data',
    type='str',
    description='Data do evento no formato de dia e mes(extenso) e ano em caso de mais de uma data para meses ou anos diferentes, separar por vírgula, ' \
    'caso não tenha nenhuma colocar como unterfuned e se for o mesmo mes e ando colocar junto os dias separando por " e "'    
)

schema_local = ResponseSchema(
    name='local',
    type='str',
    description='Cidade e país onde em caso de mais de uma, separar por vírgula, caso não tenha nenhuma colocar como unterfuned'    
)

schema_organizadores = ResponseSchema( 
    name='organizadores',
    type='list',
    description='Lista de organizadores, em caso de mais de uma, separar por vírgula, caso não tenha nenhuma colocar como unterfuned'
)

schema_temas = ResponseSchema(
    name='temas',
    type='list',
    description='Lista de temas abordados, em caso de mais de uma, separar por vírgula, caso não tenha nenhuma colocar como unterfuned'   
)

schema_ingresso = ResponseSchema(
    name='valor_ingresso',
    type='str',
    description='Preço do ingresso em moeda local, caso não tenha nenhuma colocar como unterfuned'
)

response_schema = [schema_evento,
                   schema_data,
                   schema_local,
                   schema_organizadores,
                   schema_temas,
                   schema_ingresso
                
    ]

output_parser = StructuredOutputParser.from_response_schemas(response_schema)

schema_formatado = output_parser.get_format_instructions()

print(schema_formatado)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"nome_evento": str  // Nome do evento em caso de mais de uma, separar por vírgula, caso não tenha nenhuma colocar como unterfuned
	"data": str  // Data do evento no formato de dia e mes(extenso) e ano em caso de mais de uma data para meses ou anos diferentes, separar por vírgula, caso não tenha nenhuma colocar como unterfuned e se for o mesmo mes e ando colocar junto os dias separando por " e "
	"local": str  // Cidade e país onde em caso de mais de uma, separar por vírgula, caso não tenha nenhuma colocar como unterfuned
	"organizadores": list  // Lista de organizadores, em caso de mais de uma, separar por vírgula, caso não tenha nenhuma colocar como unterfuned
	"temas": list  // Lista de temas abordados, em caso de mais de uma, separar por vírgula, caso não tenha nenhuma colocar como unterfuned
	"valor_ingresso": str  // Preço do ingresso 

In [4]:
# Enviando prompt para IA

prompt_template = ChatPromptTemplate.from_template(
""" 
Extraia do texto a seguir as seguintes informações no formato JSON:

- **nome_evento**: Nome do evento.
- **data**: Data do evento.
- **local**: Cidade e país onde ocorrerá.
- **organizadores**: Lista de organizadores.
- **temas**: Lista de temas abordados.
- **valor_ingresso**: Preço do ingresso em moeda local.

Texto: {texto}

{schema}
""", partial_variables={'schema':schema_formatado})


In [5]:
llm = ChatGoogleGenerativeAI(model='gemini-1.5-flash')

resposta = llm.invoke(prompt_template.format_messages(texto=Texto_evento))

print(resposta.content)

```json
{
  "nome_evento": "Tech Future Conference 2025",
  "data": "14 e 15 de setembro de 2025",
  "local": "Lisboa, Portugal",
  "organizadores": ["Future Minds", "InovaTech"],
  "temas": ["Inteligência Artificial", "Blockchain", "Computação Quântica", "Tecnologias Sustentáveis"],
  "valor_ingresso": "€249"
}
```


### Exemplo de Prompt para Treinar LangChain com Schema

Extraia do texto a seguir as seguintes informações no formato JSON:

- **Produto**: Produto.
- **Presente**: Verdadeiro ou Falso.
- **Sentimento**: sentimento na avaliação.
- **Positivo**: Avaliação positiva ou negativa.
- **comparação_mercado**: achou caro barato ou mesma média de valor.

Texto:

'Este soprador de folhas é bastante incrível. Ele tem quatro configurações: sopro de vela, brisa suave, cidade ventosa e tornado. ' \
'Chegou em dois dias, bem a tempo para o presente de aniversário da minha esposa. ' \
'Acho que minha esposa gostou tanto que ficou sem palavras. Até agora, fui o único a usá-lo, e tenho usado em todas as manhãs   ' \
'alternadas para limpar as folhas do nosso gramado. É um pouco mais caro do que os outros sopradores de folhas disponíveis no mercado, ' \
'mas acho que vale a pena pelas características extras.'


In [6]:
# importando libs 

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate

from dotenv import load_dotenv

from langchain.output_parsers import ResponseSchema, StructuredOutputParser

load_dotenv() # carrega arquivo .env para pegar chave API

True

In [7]:
review_text = 'Este soprador de folhas é bastante incrível. Ele tem quatro configurações: sopro de vela, brisa suave, cidade ventosa e tornado. ' \
'Chegou em dois dias, bem a tempo para o presente de aniversário da minha esposa. ' \
'Acho que minha esposa gostou tanto que ficou sem palavras. Até agora, fui o único a usá-lo, e tenho usado em todas as manhãs   ' \
'alternadas para limpar as folhas do nosso gramado. É um pouco mais caro do que os outros sopradores de folhas disponíveis no mercado, ' \
'mas acho que vale a pena pelas características extras.'

In [8]:
#Criando schemas
schema_produto = ResponseSchema(
    name='Produto',
    type='str',
    description='Nome do produto comprado, em caso de falta de informação colcoar como indefinite'
)

schema_Presente = ResponseSchema(
    name='Presente',
    type='bool',
    description='Verdadeiro se for presente' \
    'Falso se não for presente e' \
    'indefinite se não estiver a informação no texto'
)

schema_Sentimento = ResponseSchema(
    name='Sentimento',
    type='list ',
    description='coloque pelo menos 2 sentimentos da avaliação do produto'
)

schema_Positivo = ResponseSchema(
    name='Positivo',
    type='str',
    description='Forneça em uma única palavra o sentimento na avaliação do produto, seja "Positiva" ou "Negativa"'
)

schema_mercado = ResponseSchema(
    name='comparação_mercado',
    type='str',
    description='o compardor achouu o produto mais caro ou mais barato que a concorrência'
)

response_schema = [schema_produto,
                   schema_Presente,
                   schema_Sentimento,
                   schema_Positivo,
                   schema_mercado
    ]

output_parser = StructuredOutputParser.from_response_schemas(response_schema)

schema_formatado = output_parser.get_format_instructions()

print(schema_formatado)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"Produto": str  // Nome do produto comprado, em caso de falta de informação colcoar como indefinite
	"Presente": bool  // Verdadeiro se for presenteFalso se não for presente eindefinite se não estiver a informação no texto
	"Sentimento": list   // coloque pelo menos 2 sentimentos da avaliação do produto
	"Positivo": str  // Forneça em uma única palavra o sentimento na avaliação do produto, seja "Positiva" ou "Negativa"
	"comparação_mercado": str  // o compardor achouu o produto mais caro ou mais barato que a concorrência
}
```


In [9]:
#criando prompt 

review_template = ChatPromptTemplate.from_template(
''' 
Extraia do texto a seguir as seguintes informações no formato JSON:

- **Produto**: Produto.
- **Presente**: Verdadeiro ou Falso.
- **Sentimento**: sentimento na avaliação.
- **Positivo**: Avaliação positiva ou negativa.
- **comparação_mercado**: achou caro barato ou mesma média de valor.

Texyo: {review}

{schema}
''' , partial_variables={'schema': schema_formatado}
)

In [10]:
#Chamando IA com retorno

llm = ChatGoogleGenerativeAI(model='gemini-1.5-flash')

retorno = llm.invoke(review_template.format_messages(review=review_text))

print(retorno.content)

```json
{
  "Produto": "soprador de folhas",
  "Presente": true,
  "Sentimento": ["incrível", "satisfeito"],
  "Positivo": "Positiva",
  "comparação_mercado": "mais caro"
}
```
